In [107]:
import pyodbc
import pandas as pd

# Figure out which sprocs are missing

## SQL Client

In [108]:
class Client(object):

    DRIVER = '{ODBC Driver 18 for SQL Server}'

    def __init__(self, server, database, username, password):
        self.conn = Client._connect(server, database, username, password)

    @staticmethod
    def _connect(server, database, username, password):
        connection_string = Client._get_connection_string(server, database, username, password)
        return pyodbc.connect(connection_string)

    @staticmethod
    def _get_connection_string(server, database, username, password):
        return f'DRIVER={Client.DRIVER};SERVER={server};DATABASE={database};UID={username};PWD={password}'

    def _execute(self, query):
        cursor = self.conn.cursor()
        cursor.execute(query)
        return cursor
        
    def get_all_query_results(self, query):
        cursor = self._execute(query)
        return cursor.fetchall()


In [109]:
TARGET_SERVER = 'sql-sqldbpoc-dev-centralus-lacewing.database.windows.net'
TARGET_DATABASE = 'IPEP_C3LX_Training'
TARGET_USERNAME = 'sql-sqldbpoc-dev-centralus-lacewing-admin'
TARGET_PASSWORD = ')D&{F3T*(51_@8*4-mOp'

In [110]:
target_client = Client(TARGET_SERVER, TARGET_DATABASE, TARGET_USERNAME, TARGET_PASSWORD)

## List sprocs in target database

In [111]:
LIST_SPROCS = """
SELECT routines.SPECIFIC_SCHEMA, routines.SPECIFIC_NAME, modules.definition
FROM sys.sql_modules modules
join INFORMATION_SCHEMA.ROUTINES routines
on modules.object_id = object_id(routines.SPECIFIC_SCHEMA + '.' + routines.SPECIFIC_NAME)
"""

In [112]:
results = target_client.get_all_query_results(LIST_SPROCS)

In [113]:
target_df = pd.DataFrame.from_records(results)

In [114]:
target_df.columns = ['schema', 'sproc', 'ddl']

In [115]:
target_df

,schema,sproc,ddl
0,dbo,USP_InsertInto_Trn_Patient_Med_Dosage,\nCreate procedure USP_InsertInto_Trn_Patient_...
1,dbo,SP_Mst_PCP_Search,\nCreate Procedure [dbo].[SP_Mst_PCP_Search] (...
2,dbo,USP_InsertIntoDRGListFromTempTable,\nCREATE PROCEDURE USP_InsertIntoDRGListFromTe...
3,dbo,SP_Mst_Pharmacy_Search,\nCreate procedure [dbo].[SP_Mst_Pharmacy_Sear...
4,dbo,USP_InsertOrUpdateUploadSummary,\nCreate procedure USP_InsertOrUpdateUploadSum...
...,...,...,...
241,dbo,USP_InsertInto_Mst_Prescriber,\nCreate procedure USP_InsertInto_Mst_Prescrib...
242,dbo,SP_Mst_Medicine_Search,\nCreate procedure SP_Mst_Medicine_Search (@Se...
243,dbo,USP_InsertInto_MstPatient,\nCreate procedure USP_InsertInto_MstPatient (...
244,dbo,SP_Mst_MedNDC,\nCreate procedure SP_Mst_MedNDC AS\r\nBEGIN\r...


## Read exported data from source database (database too old to be queried using odbc)

In [116]:
source_df = pd.read_csv('./ipep-source-db-sprocs.csv', header=None)
source_df.columns = ['schema', 'sproc', 'ddl']

In [117]:
source_df

,schema,sproc,ddl
0,dbo,USP_Get_UHCNDC_MedList,Create procedure USP_Get_UHCNDC_MedList AS B...
1,dbo,USP_Trn_Patient_Pharmcy,Create procedure USP_Trn_Patient_Pharmcy (@Pa...
2,dbo,USP_GetAccountWiseActivityList,Create procedure [dbo].[USP_GetAccountWiseAct...
3,dbo,USP_Trn_Patient_Prescriber,Create procedure USP_Trn_Patient_Prescriber (...
4,dbo,USP_GetAllBillingCodeFees,Create procedure USP_GetAllBillingCodeFees A...
...,...,...,...
240,dbo,USP_ServiceBillingSummaryReport,CREATE procedure USP_ServiceBillingSummaryRep...
241,dbo,USP_Get_All_Trn_BookAppointment,create procedure USP_Get_All_Trn_BookAppointm...
242,dbo,USP_Trn_Patient_Contact,Create procedure USP_Trn_Patient_Contact (@Pa...
243,dbo,USP_Get_Trn_BookAppointment,create procedure USP_Get_Trn_BookAppointment ...


In [118]:
missing_sprocs = set(source_df.sproc).difference(set(target_df.sproc))

In [119]:
missing_df = source_df[source_df.sproc.isin(missing_sprocs)]    

## We are missing 14 sprocs

In [120]:
missing_df.shape

(0, 3)

# Try to migrate by executing ddl

## Write ddl to file so we can sqlfluff it

In [88]:
# for index, row in missing_df.iterrows():
#     with open(f"./ddl/{row['sproc']}.sql", 'w') as file:
#         file.write(row['ddl'])

## Try creating as many of the existing sprocs as possible

In [91]:
exceptions = []
for index, row in missing_df.iterrows():
    print(f"./ssms-script-ddl/dbo.{row['sproc']}.StoredProcedure.sql")
    with open(f"./ssms-script-ddl/dbo.{row['sproc']}.StoredProcedure.sql", 'r', encoding='utf-16') as file:
        ddl = file.read()

    try:
        target_client.get_all_query_results(ddl)
    except Exception as e:
        print("*"*100)
        print(e)
        print(row.ddl)
        exceptions.append(row)

./ssms-script-ddl/dbo.USP_Get_UHCNDC_MedList.StoredProcedure.sql
****************************************************************************************************
('42000', "[42000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Incorrect syntax near 'GO'. (102) (SQLExecDirectW)")
 Create procedure USP_Get_UHCNDC_MedList AS  BEGIN 	-- We need unique NDC from Trn_UHCMedNDC_List so we are taking distinct NDC. 	SELECT distinct NDC.NDC,NDC.Med_ID,NDC.Strength_ID  	FROM TRN_UHCMedNDC_List UHC INNER JOIN Mst_MedNDC NDC ON UHC.NDC= NDC.NDC END --EXEC USP_Get_UHCNDC_MedList
./ssms-script-ddl/dbo.USP_Trn_Patient_Pharmcy.StoredProcedure.sql
****************************************************************************************************
('42000', "[42000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Incorrect syntax near 'GO'. (102) (SQLExecDirectW)")
 Create procedure USP_Trn_Patient_Pharmcy (@PatientId INT) AS BEGIN 	SELECT  		TPP.ID, 		Patient_ID, 		MP.Id AS Pharmacy_ID,

In [90]:
len(exceptions)

158

## Check to see how many new sprocs we got (not that many)

In [70]:
results2 = target_client.get_all_query_results(LIST_SPROCS)

In [71]:
len(results), len(results2)

(87, 87)

In [72]:
missing_df.to_csv('sprocs-not-migrated.csv')